In [2]:
import pandas as pd
pd.options.display.max_columns = 999
import pgeocode as geo
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [3]:
sc= pd.read_excel(r'Input\\social_capital.97-05-09.updated8.28.17.xlsx', sheet_name='2009')
sc.head(1)

,fips,areaname,relig09,civic09,bus09,pol09,prof09,labor09,bowl09,fitns09,golf09,sport09,pop09,respn10,pvote08,nccs09,assn09,sk09
0,1001,"Autauga, AL",50,7,3,0,1,5,1,4,2,0,54135,0.78,0.635648,182.0,1.348481,-0.32673


In [4]:
cols_rename= {
            'areaname':'area',
            'County_Name':'area',
            'fitns09': 'fit09',
            }

In [5]:
sc.describe()

,fips,relig09,civic09,bus09,pol09,prof09,labor09,bowl09,fitns09,golf09,sport09,pop09,respn10,pvote08,nccs09,assn09,sk09
count,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3108.000000,3.108000e+03,3108.000000,3108.000000,3106.000000,3108.000000,3108.000000
mean,30685.834299,57.781853,9.028314,5.428893,0.725869,2.214286,4.864865,1.394144,9.703990,3.791828,0.255148,9.802803e+04,0.705502,0.588928,494.433677,1.410900,-0.000005
std,14982.915779,123.284644,21.995563,16.682472,3.857889,10.555466,15.196865,2.963905,30.062512,7.311064,1.103972,3.121815e+05,0.111264,0.095851,1493.509124,0.709241,1.338274
min,1001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.700000e+01,0.000000,0.160159,1.000000,0.000000,-3.925233
25%,19046.500000,11.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.135600e+04,0.640000,0.527927,66.000000,0.968246,-0.834727
50%,29214.000000,24.000000,3.000000,2.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,0.000000,2.594050e+04,0.740000,0.592295,141.000000,1.283137,-0.220431
75%,46009.500000,54.000000,8.000000,4.000000,0.000000,1.000000,3.000000,2.000000,6.000000,4.000000,0.000000,6.675225e+04,0.790000,0.651958,339.000000,1.682082,0.526460
max,56045.000000,3258.000000,538.000000,474.000000,133.000000,318.000000,292.000000,58.000000,738.000000,142.000000,29.000000,9.787400e+06,0.950000,2.078947,41125.000000,6.821282,17.440505


In [6]:
sc.rename(columns=cols_rename, inplace=True)
sc.columns= sc.columns.str.strip().str.lower()
sc.columns=sc.columns.str.replace('\d+','',regex=True).str.strip()
sc['nccs']= sc['nccs'].replace(np.nan, 0) # This is one of the key things that make results: Before this fix: corr= 0.54. After this fix corr= 0.99

sc['nccs']= sc['nccs']/(sc['pop']/1000)
sc['assn']= (sc['relig'] + sc['civic'] + sc['bus'] + sc['pol'] + sc['prof'] + sc['labor'] + sc['bowl'] + sc['fit'] + sc['golf'] + sc['sport']) / (sc['pop']/1000)

sc.drop(columns=['bowl', 'civic', 'fit', 'golf', 'relig', 'sport', 'pol', 'prof', 'bus', 'labor'], inplace=True)
sc.drop(columns=['pop'],inplace=True)
sc.head(3)

,fips,area,respn,pvote,nccs,assn,sk
0,1001,"Autauga, AL",0.78,0.635648,3.361965,1.348481,-0.326730
1,1003,"Baldwin, AL",0.73,0.608996,4.108001,1.226269,-0.422958
2,1005,"Barbour, AL",0.63,0.512425,3.868822,0.867773,-1.196517


In [7]:
x = sc.loc[:, ['respn','pvote','assn','nccs']].values
# Separating out the target
y = sc.loc[:,['fips']].values
# Standardizing the features
x = StandardScaler().fit_transform(x)

pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
pca_df = pd.DataFrame(data = principalComponents
             , columns = ['pc1'])
pca_df.head(2)

,pc1
0,-0.326766
1,-0.423096


In [8]:
sc['pc1']=pca_df[['pc1']]

In [9]:
sc.head()

,fips,area,respn,pvote,nccs,assn,sk,pc1
0,1001,"Autauga, AL",0.78,0.635648,3.361965,1.348481,-0.326730,-0.326766
1,1003,"Baldwin, AL",0.73,0.608996,4.108001,1.226269,-0.422958,-0.423096
2,1005,"Barbour, AL",0.63,0.512425,3.868822,0.867773,-1.196517,-1.196345
3,1007,"Bibb, AL",0.58,0.522517,2.571815,1.307702,-0.951097,-0.951338
4,1009,"Blount, AL",0.80,0.555639,2.110183,0.906855,-1.303935,-1.303778


In [10]:
sc['sk'].corr(sc['pc1'])

0.9999999870910082

In [11]:
sc.shape

(3108, 8)

In [12]:
sc['year']=2009
sc.to_csv("output\\sc09.csv")